In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


In [17]:
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

## Filtering column "mail_type"
train_x = train_df[['org', 'tld', 'ccs', 'bcced', 'mail_type', 'images', 'urls', 'salutations', 'designation', 'chars_in_subject', 'chars_in_body']]
train_x[['org', 'tld', 'mail_type']] = train_x[['org', 'tld', 'mail_type']].fillna(value='None')
train_x[['chars_in_subject']] = train_x[['chars_in_subject']].fillna(value='0')
train_x[['chars_in_subject']] = train_x[['chars_in_subject']].astype('int')
train_y = train_df[['label']]

test_x = test_df[['org', 'tld', 'ccs', 'bcced', 'mail_type', 'images', 'urls', 'salutations', 'designation', 'chars_in_subject', 'chars_in_body']]
test_x = test_x.fillna(value='None')



In [18]:
train_x.describe()

,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body
count,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,8.017600e+04
mean,0.473259,0.002869,7.875998,34.433172,0.406381,0.104246,51.524970,1.971784e+05
std,3.178805,0.053484,305.444093,97.904682,0.491160,0.305581,33.722776,1.890130e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000e+00
25%,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,32.000000,4.651000e+03
50%,0.000000,0.000000,2.000000,17.000000,0.000000,0.000000,42.000000,1.999950e+04
75%,0.000000,0.000000,8.000000,44.000000,1.000000,0.000000,62.000000,4.519300e+04
max,179.000000,1.000000,83481.000000,21540.000000,1.000000,1.000000,606.000000,7.438108e+07


In [19]:
train_x.head()

,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body
0,reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49,80027
1,edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107,2961
2,usebackpack,com,0,0,text/html,4,17,0,0,35,25149
3,granular,ai,0,0,multipart/mixed,0,0,0,0,15,635296
4,github,com,1,0,multipart/alternative,2,11,0,0,49,2355


In [28]:
## Do one hot encoding of categorical feature
feat_enc = OneHotEncoder()

# Get list of categorical variables
s = (train_x.dtypes == 'object')
object_cols = list(s[s].index) 
print(object_cols)

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_x[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(test_x[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = train_x.index
OH_cols_valid.index = test_x.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_x.drop(object_cols, axis=1)
num_X_valid = test_x.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

OH_X_train.head()

['org', 'tld', 'mail_type']


,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,0,1,...,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344
0,0,0,35,120,0,0,49,80027,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,7,0,0,107,2961,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,4,17,0,0,35,25149,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,15,635296,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,2,11,0,0,49,2355,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
## Train a simple KNN classifier using featurized data
model = DecisionTreeRegressor(random_state=1)
model.fit(train_x, train_y)
pred_y = model.predict(test_x_featurized)

ValueError: could not convert string to float: 'reply'

In [13]:
## Save results to submission file
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("knn_sample_submission.csv", index=True, index_label='Id')

In [18]:
#rf_val_mae = mean_absolute_error(model.predict(test_x_featurized), test_y)